### Basic Data Fetch Prototypes

Thie first one I found on the blog I recently published an article on:
https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859

Feel free to use the SDK API token I setup referenced in the `noaa` class.

In [8]:
import requests, pandas as pd, numpy as np, json
#parse the datetimes we get from NOAA
from datetime import datetime



#Long Beach Airport station
station_id = 'GHCND:USW00023129'

In [41]:
class noaa:
    
    base_url   = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate=2015-01-01&enddate=2015-12-31"
    noaa_token = 'gqfRhLiqfjECEAjlenVDCVImuElAuPGA'
    
    def __init__(self, **opts):
        for attr, value in opts.items():
            if hasattr(self, attr):
                setattr(self, attr, value)
        
    def request_json(self, url, **params):
        headers = {'token': self.noaa_token}
        result = requests.get(url, headers = headers)
        return result.json()
    
    def get_station(self, datasetid = "GHCND", datatypeid = "TAVG", stationid = "GHCND:USW00023129", startdate = "2015-01-01", enddate = "2015-12-31"):
        return self.request_json(self.base_url)
        
w = noaa()
response = w.get_station()

In [38]:
df = pd.DataFrame(response['results'])
df['date'] = pd.to_datetime(df['date'])

### From the noaa-sdk

Someone on github published a package in teh pypi universe system.  Here's an example of its usage (uncomment the next line to install the package).

https://pypi.org/project/noaa-sdk/

In [39]:
#!pip install noaa_sdk

     |████████████████████████████████| 204kB 11.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 7.6MB/s  eta 0:00:01
  Created wheel for noaa-sdk: filename=noaa_sdk-0.1.17-cp37-none-any.whl size=12241 sha256=0541e51b0b5455b0a2eaab3f089903cca9d20f676698fb77a1a3fa0b9dac9bde
  Stored in directory: /Users/davidyerrington/Library/Caches/pip/wheels/23/b6/1d/6445bdc059d7d3653c654f05fab68b1017c644020a3f65eec2
  Created wheel for httplib2: filename=httplib2-0.10.3-cp37-none-any.whl size=83988 sha256=28162f14614b2ad65dead4ff77ab7e9671e37b4bbcd05bb544a7c9623bfb99e1
  Stored in directory: /Users/davidyerrington/Library/Caches/pip/wheels/d6/f4/9c/f4eab4c19c0bde393b00a1f83afe12cc469852ff3810cd6f6d
Successfully built noaa-sdk httplib2
  Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0


### Lookup Weather by Zipcode

The return format is a generator which is a type not compatible with DataFrame inputs so we will convert each obervation to a list for quick view.  I do believe we could create an easy mapping of this data that could automatically translate to a nice convention like `feature_name_unit_type`.

> The service seems to work great, except for when you plug seldovia zipcode in. Will try to lookup by station next.

In [53]:
from noaa_sdk import noaa
n = noaa.NOAA()
observations = n.get_observations('11365','US') # 99663 = seldovia, but it seems service doesn't respond
data = []
for row in observations:
    data.append(row)

#### Observations to DataFrame

Much of the observations are still encoded as dictionaries (not a bad thing).  I can write an apply function to encode these as new features while saving the old ones.  If we like this data, we can deal with it before we put it in DataFrames.

In [65]:
df_noaa = pd.DataFrame(data)
df_noaa['timestamp'] = pd.to_datetime(df_noaa['timestamp'])

In [67]:
df_noaa.head()

,@id,@type,elevation,station,timestamp,rawMessage,textDescription,icon,presentWeather,temperature,...,visibility,maxTemperatureLast24Hours,minTemperatureLast24Hours,precipitationLastHour,precipitationLast3Hours,precipitationLast6Hours,relativeHumidity,windChill,heatIndex,cloudLayers
0,https://api.weather.gov/stations/KLGA/observat...,wx:ObservationStation,"{'value': 9, 'unitCode': 'unit:m'}",https://api.weather.gov/stations/KLGA,2020-02-12 01:51:00+00:00,KLGA 120151Z 30013KT 10SM FEW021 BKN100 OVC110...,Cloudy,https://api.weather.gov/icons/land/night/ovc?s...,[],"{'value': 7.800000000000011, 'unitCode': 'unit...",...,"{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 70.6580455433216, 'unitCode': 'unit:...","{'value': 4.208512575318366, 'unitCode': 'unit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","[{'base': {'value': 640, 'unitCode': 'unit:m'}..."
1,https://api.weather.gov/stations/KLGA/observat...,wx:ObservationStation,"{'value': 9, 'unitCode': 'unit:m'}",https://api.weather.gov/stations/KLGA,2020-02-12 01:31:00+00:00,KLGA 120131Z 31014KT 10SM SCT021 OVC110 08/04 ...,Cloudy,https://api.weather.gov/icons/land/night/ovc?s...,[],"{'value': 7.800000000000011, 'unitCode': 'unit...",...,"{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 76.3607448618594, 'unitCode': 'unit:...","{'value': 4.049072026674992, 'unitCode': 'unit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","[{'base': {'value': 640, 'unitCode': 'unit:m'}..."
2,https://api.weather.gov/stations/KLGA/observat...,wx:ObservationStation,"{'value': 9, 'unitCode': 'unit:m'}",https://api.weather.gov/stations/KLGA,2020-02-12 00:58:00+00:00,KLGA 120058Z 31009KT 10SM -RA FEW019 BKN026 OV...,Light Rain,https://api.weather.gov/icons/land/night/rain?...,"[{'intensity': 'light', 'modifier': None, 'wea...","{'value': 7.800000000000011, 'unitCode': 'unit...",...,"{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 79.08482210318559, 'unitCode': 'unit...","{'value': 5.0123538466526725, 'unitCode': 'uni...","{'value': None, 'unitCode': 'unit:degC', 'qual...","[{'base': {'value': 580, 'unitCode': 'unit:m'}..."
3,https://api.weather.gov/stations/KLGA/observat...,wx:ObservationStation,"{'value': 9, 'unitCode': 'unit:m'}",https://api.weather.gov/stations/KLGA,2020-02-12 00:51:00+00:00,KLGA 120051Z 31011KT 10SM SCT022 BKN041 OVC050...,Cloudy,https://api.weather.gov/icons/land/night/ovc?s...,[],"{'value': 7.800000000000011, 'unitCode': 'unit...",...,"{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 79.08482210318559, 'unitCode': 'unit...","{'value': 4.55996919337781, 'unitCode': 'unit:...","{'value': None, 'unitCode': 'unit:degC', 'qual...","[{'base': {'value': 670, 'unitCode': 'unit:m'}..."
4,https://api.weather.gov/stations/KLGA/observat...,wx:ObservationStation,"{'value': 9, 'unitCode': 'unit:m'}",https://api.weather.gov/stations/KLGA,2020-02-11 23:51:00+00:00,KLGA 112351Z 30005KT 10SM OVC060 08/05 A2982

### Encoding to new features is actually not that bad

This function will translate the lower level format as new columns on select columns.  I've labeld them below as `to_encode`.  In this example, we use an `apply` function on our data which allows you to run the given function on every column (axis = 0 (default)), or rows (axis = 1).  This is preferable to basic Python iteration because Pandas optimizes these operations against its own unique "special" data types under the hood.  

I have a few tutorials that I wrote on this type of feature within Pandas and I can upload it somewhere if you let me know.

> Protip:  When in Pandas, use `.map` and `.apply` on your DataFrames to make changes rather than a for loop or other iterators.

In [126]:
to_encode = ['elevation', 'temperature', 'visibility', 'minTemperatureLast24Hours', 'precipitationLastHour', 'precipitationLast3Hours', 'precipitationLast6Hours', 'relativeHumidity', 'windChill', 'heatIndex', 'cloudLayers']

def encode_value_to_unit(row):
    for column in row.index:
        try:
            row[f"{column}_{row[column]['unitCode']}"] = row[column]['value']
        except:
            print(f"Couldn't encode feature for {column}")
    return row

    
df_noaa_encoded = df_noaa[to_encode].apply(encode_value_to_unit, axis = 1)

Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers


### Still cruft in the DataFrame

New features were added to the end and they look pretty clean.

In [128]:
df_noaa_encoded.head()

,elevation,temperature,visibility,minTemperatureLast24Hours,precipitationLastHour,precipitationLast3Hours,precipitationLast6Hours,relativeHumidity,windChill,heatIndex,...,elevation_unit:m,temperature_unit:degC,visibility_unit:m,minTemperatureLast24Hours_unit:degC,precipitationLastHour_unit:m,precipitationLast3Hours_unit:m,precipitationLast6Hours_unit:m,relativeHumidity_unit:percent,windChill_unit:degC,heatIndex_unit:degC
0,"{'value': 9, 'unitCode': 'unit:m'}","{'value': 7.800000000000011, 'unitCode': 'unit...","{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 70.6580455433216, 'unitCode': 'unit:...","{'value': 4.208512575318366, 'unitCode': 'unit...","{'value': None, 'unitCode': 'unit:degC', 'qual...",...,9,7.8,16090,None,0.0,NaN,NaN,70.658046,4.208513,NaN
1,"{'value': 9, 'unitCode': 'unit:m'}","{'value': 7.800000000000011, 'unitCode': 'unit...","{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 76.3607448618594, 'unitCode': 'unit:...","{'value': 4.049072026674992, 'unitCode': 'unit...","{'value': None, 'unitCode': 'unit:degC', 'qual...",...,9,7.8,16090,None,0.0,NaN,NaN,76.360745,4.049072,NaN
2,"{'value': 9, 'unitCode': 'unit:m'}","{'value': 7.800000000000011, 'unitCode': 'unit...","{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 79.08482210318559, 'unitCode': 'unit...","{'value': 5.0123538466526725, 'unitCode': 'uni...","{'value': None, 'unitCode': 'unit:degC', 'qual...",...,9,7.8,16090,None,0.0,NaN,NaN,79.084822,5.012354,NaN
3,"{'value': 9, 'unitCode': 'unit:m'}","{'value': 7.800000000000011, 'unitCode': 'unit...","{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 79.08482210318559, 'unitCode': 'unit...","{'value': 4.55996919337781, 'unitCode': 'unit:...","{'value': None, 'unitCode': 'unit:degC', 'qual...",...,9,7.8,16090,None,NaN,NaN,NaN,79.084822,4.559969,NaN
4,"{'value': 9, 'unitCode': 'unit:m'}","{'value': 8.300000000000011, 'unitCode': 'unit...","{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 0.0005, 'unitCode': 'unit:m', 'quali...","{'value': 79.71241189231577, 'unitCode': 'unit...","{'value': 6.737267429777944, 'unitCode': 'unit...","{'value': None, 'unitCode': 'unit:degC', 'qual...",...,9,8.3,16090,None,NaN,NaN,0.0005,79.712412,6.737267,NaN


### Save encoded data to file for review

In [129]:
df_noaa.to_csv("../data/processed/noaa_from_api.csv", encoding = "UTF8")